## Create a simple forecast from the NDFD which can be used with NWS graphics

In [1]:
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy
from metpy.units import units
from pyproj import Proj
import pytz
from pytz import timezone
from PIL import Image

### Get current time rounded down to last 30 minute interval

In [2]:
def rounded_to_the_last_30_minute():
    now = datetime.now()
    rounded = now - (now - datetime.min) % timedelta(hours=1)
    return rounded

In [3]:
date = rounded_to_the_last_30_minute()

### Import NDFD data for latest time

In [4]:
YYYYMMDD_HHMM = date.strftime('%Y%m%d_%H%M')

In [5]:
File = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_"+YYYYMMDD_HHMM+".grib2"
File

'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_20240223_1800.grib2'

In [6]:
ds = xr.open_dataset(File)

### Parse for data map projection and add lats & lons

In [7]:
ds = ds.metpy.parse_cf()
ds = ds.metpy.assign_latitude_longitude(force=False)
ds

<xarray.Dataset>
Dimensions:                                                                   (
                                                                               time: 6,
                                                                               time_bounds_1: 2,
                                                                               time2: 7,
                                                                               time2_bounds_1: 2,
                                                                               time4: 13,
                                                                               ...
                                                                               height_above_ground1: 1,
                                                                               x: 2145,
                                                                               y: 1377,
                                                                               time5: 54,
                                                                               height_above_ground: 1,
                                                                               time3: 42)
Coordinates: (12/15)
    reftime                                                                   datetime64[ns] ...
  * time                                                                      (time) datetime64[ns] ...
  * time2                                                                     (time2) datetime64[ns] ...
  * time4                                                                     (time4) datetime64[ns] ...
  * time6                                                                     (time6) datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
    ...                                                                        ...
    metpy_crs                                                                 object ...
  * time5                                                                     (time5) datetime64[ns] ...
  * height_above_ground                                                       (height_above_ground) float32 ...
  * time3                                                                     (time3) datetime64[ns] ...
    latitude                                                                  (y, x) float64 ...
    longitude                                                                 (y, x) float64 ...
Dimensions without coordinates: time_bounds_1, time2_bounds_1, time4_bounds_1,
                                time6_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time_bounds                                                               (time, time_bounds_1) datetime64[ns] ...
    time2_bounds                                                              (time2, time2_bounds_1) datetime64[ns] ...
    time4_bounds                                                              (time4, time4_bounds_1) datetime64[ns] ...
    time6_bounds                                                              (time6, time6_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time1, height_above_ground1, y, x) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time6, y, x) float32 ...
    Apparent_Temperature_height_above_ground                                  (time1, height_above_ground1, y, x) float32 ...
    Weather_string_surface                                                    (time1, y, x) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time1, height_above_ground, y, x) float32 ...
    Wind_speed_height_above_ground                                           

In [8]:
x, y = ds.x, ds.y

### Define max temp variable & function to get closest gridpoint to ETEC

In [9]:
def find_closest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx

### Get gridpoint closest to ETEC

In [10]:
proj_data = ds.Temperature_height_above_ground.metpy.cartopy_crs
proj_data;

pFull = Proj(proj_data)

In [11]:
siteName = "ETEC"
siteLat, siteLon = (42.75, -73.80) #lat & lon of gridpoint over ETEC
siteX, siteY = pFull(siteLon, siteLat)
siteXidx, siteYidx = find_closest(x, siteX), find_closest(y, siteY)

In [12]:
ds = ds.isel(x = siteXidx, y = siteYidx).isel()
ds

<xarray.Dataset>
Dimensions:                                                                   (
                                                                               time: 6,
                                                                               time_bounds_1: 2,
                                                                               time2: 7,
                                                                               time2_bounds_1: 2,
                                                                               time4: 13,
                                                                               ...
                                                                               time6_bounds_1: 2,
                                                                               time1: 58,
                                                                               height_above_ground1: 1,
                                                                               time5: 54,
                                                                               height_above_ground: 1,
                                                                               time3: 42)
Coordinates: (12/15)
    reftime                                                                   datetime64[ns] ...
  * time                                                                      (time) datetime64[ns] ...
  * time2                                                                     (time2) datetime64[ns] ...
  * time4                                                                     (time4) datetime64[ns] ...
  * time6                                                                     (time6) datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
    ...                                                                        ...
    metpy_crs                                                                 object ...
  * time5                                                                     (time5) datetime64[ns] ...
  * height_above_ground                                                       (height_above_ground) float32 ...
  * time3                                                                     (time3) datetime64[ns] ...
    latitude                                                                  float64 ...
    longitude                                                                 float64 ...
Dimensions without coordinates: time_bounds_1, time2_bounds_1, time4_bounds_1,
                                time6_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time_bounds                                                               (time, time_bounds_1) datetime64[ns] ...
    time2_bounds                                                              (time2, time2_bounds_1) datetime64[ns] ...
    time4_bounds                                                              (time4, time4_bounds_1) datetime64[ns] ...
    time6_bounds                                                              (time6, time6_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time1, height_above_ground1) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time6) float32 ...
    Apparent_Temperature_height_above_ground                                  (time1, height_above_ground1) float32 ...
    Weather_string_surface                                                    (time1) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time1, height_above_ground) float32 ...
    Wind_speed_height_above_ground                                            (time1, height_above_ground) fl

In [13]:
temp = ds.Temperature_height_above_ground
timeDim, vertDim = temp.metpy.time.name, temp.metpy.vertical.name
idxVert = 0 # First (and in this case, only) vertical level
idxTime = slice(None, 24)
vertDict = {vertDim: idxVert}
timeDict = {timeDim: idxTime}
temp = temp.isel(vertDict)
temp = temp.isel(timeDict)
temp = temp.drop_vars(['reftime', 'x', 'y', 'metpy_crs', 'longitude', 'latitude', vertDim])
df_temp = temp.to_dataframe()

In [14]:
temp

<xarray.DataArray 'Temperature_height_above_ground' (time1: 24)>
[24 values with dtype=float32]
Coordinates:
  * time1    (time1) datetime64[ns] 2024-02-23T19:00:00 ... 2024-02-24T18:00:00
Attributes: (12/13)
    long_name:                       Temperature @ Specified height level abo...
    units:                           K
    description:                     Temperature
    grid_mapping:                    LambertConformal_Projection
    Grib_Variable_Id:                VAR_0-0-0_L103
    Grib2_Parameter:                 [0 0 0]
    ...                              ...
    Grib2_Parameter_Category:        Temperature
    Grib2_Parameter_Name:            Temperature
    Grib2_Level_Type:                103
    Grib2_Level_Desc:                Specified height level above ground
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  UnknownStatType--1

In [15]:
dewp = ds.Dewpoint_temperature_height_above_ground
dewp = dewp.isel(vertDict)
dewp = dewp.isel(timeDict)
dewp = dewp.drop_vars(['reftime', 'x', 'y', 'metpy_crs', 'longitude', 'latitude', vertDim])
df_dewp = dewp.to_dataframe()

In [16]:
dewp

<xarray.DataArray 'Dewpoint_temperature_height_above_ground' (time1: 24)>
[24 values with dtype=float32]
Coordinates:
  * time1    (time1) datetime64[ns] 2024-02-23T19:00:00 ... 2024-02-24T18:00:00
Attributes: (12/13)
    long_name:                       Dewpoint temperature @ Specified height ...
    units:                           K
    description:                     Dewpoint temperature
    grid_mapping:                    LambertConformal_Projection
    Grib_Variable_Id:                VAR_0-0-6_L103
    Grib2_Parameter:                 [0 0 6]
    ...                              ...
    Grib2_Parameter_Category:        Temperature
    Grib2_Parameter_Name:            Dewpoint temperature
    Grib2_Level_Type:                103
    Grib2_Level_Desc:                Specified height level above ground
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  UnknownStatType--1

In [17]:
wdsp = ds.Wind_speed_height_above_ground
vertDimWind = wdsp.metpy.vertical.name
vertDictWind = {vertDimWind: idxVert}
wdsp = wdsp.isel(vertDictWind)
wdsp = wdsp.isel(timeDict)
wdsp = wdsp.drop_vars(['reftime', 'x', 'y', 'metpy_crs', 'longitude', 'latitude', vertDimWind])
wdsp = wdsp * 2.23694
df_wdsp = wdsp.to_dataframe()

In [18]:
wdsp

<xarray.DataArray 'Wind_speed_height_above_ground' (time1: 24)>
array([ 4.6975737,  4.6975737,  5.8160434,  5.8160434,  4.6975737,
        4.6975737,  4.6975737,  5.8160434,  6.9345136,  6.9345136,
        8.052983 , 10.289924 , 11.408393 , 11.408393 , 11.408393 ,
       12.750557 , 12.750557 , 11.408393 , 11.408393 , 11.408393 ,
       11.408393 , 12.750557 , 13.869027 , 13.869027 ], dtype=float32)
Coordinates:
  * time1    (time1) datetime64[ns] 2024-02-23T19:00:00 ... 2024-02-24T18:00:00

In [19]:
df_merge1 = pd.merge(df_temp, df_dewp, on=timeDim)
df = pd.merge(df_merge1, df_wdsp, on=timeDim)
df

,Temperature_height_above_ground,Dewpoint_temperature_height_above_ground,Wind_speed_height_above_ground
time1,,,
2024-02-23 19:00:00,280.899994,274.799988,4.697574
2024-02-23 20:00:00,281.500000,274.799988,4.697574
2024-02-23 21:00:00,281.500000,274.799988,5.816043
2024-02-23 22:00:00,280.899994,274.299988,5.816043
2024-02-23 23:00:00,279.299988,274.299988,4.697574
2024-02-24 00:00:00,278.100006,273.700012,4.697574
2024-02-24 01:00:00,277.000000,273.100006,4.697574
2024-02-24 02:00:00,277.000000,272.600006,5.816043
2024-02-24 03:00:00,276.500000,272.000000,6.934514


In [20]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, AutoDateLocator, YearLocator, HourLocator, DayLocator, MonthLocator

from netCDF4 import num2date

from metpy.units import units
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS
from datetime import datetime, timedelta

import pandas as pd
import xarray as xr
import metpy
import metpy.calc as mpcalc
from PIL import Image
import pytz
from pytz import timezone

In [21]:
# Albany version is GEMPAK converted to netCDF.
# Two possibilities:  one is the one-year archive, updated once per day; the other is the most-recent week archive, updated in real time.
#metar_cat_url = 'http://thredds.atmos.albany.edu:8080/thredds/catalog/metarArchive/ncdecoded/catalog.xml?dataset=metarArchive/ncdecoded/Archived_Metar_Station_Data_fc.cdmr'
metar_cat_url = 'http://thredds.atmos.albany.edu:8080/thredds/catalog/metar/ncdecoded/catalog.xml?dataset=metar/ncdecoded/Metar_Station_Data_fc.cdmr'
# Parse the xml and return a THREDDS Catalog Object.
catalog = TDSCatalog(metar_cat_url)

metar_dataset = catalog.datasets['Feature Collection']

In [22]:
ncss_url = metar_dataset.access_urls['NetcdfSubset']

In [23]:
# We have the URL for our catalog's NetCDF Subset service, now create an object using the ncss client and pull
ncss = NCSS(ncss_url)

In [24]:
ncss.variables.remove('_isMissing')

In [25]:
# get current date and time

now = datetime.utcnow()
now = datetime(now.year, now.month, now.day, now.hour)
day_1 = now - timedelta(hours = 23, minutes = 30)

# build the query
query = ncss.query()

In [26]:
# Select a location or list of locatons. 
#This can be either a single point (THREDDS will attempt to locate the nearest station) or an actual METAR site ID.

query.add_query_parameter(stns='ALB',subset='stns')

query.time_range(day_1, now)

#query.variables('all')
query.variables('PMSL', 'TMPC', 'DWPC', 'WNUM',
                'DRCT', 'SKNT', 'GUST', 'ALTI', 'CHC1', 'CHC2', 'CHC3')
query.accept('netcdf')

var=DWPC&var=PMSL&var=CHC2&var=TMPC&var=SKNT&var=ALTI&var=CHC1&var=CHC3&var=GUST&var=WNUM&var=DRCT&time_start=2024-02-22T18%3A30%3A00&time_end=2024-02-23T18%3A00%3A00&stns=ALB&subset=stns&accept=netcdf

In [27]:
data = ncss.get_data(query)

In [28]:
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.6
    history: Written by CFPointWriter
    title: Extracted data from TDS Feature Collection Metar Station Data
    time_coverage_start: 2024-02-22T18:40:00Z
    time_coverage_end: 2024-02-23T18:00:00Z
    geospatial_lat_min: 42.7495
    geospatial_lat_max: 42.7505
    geospatial_lon_min: -73.80050305175781
    geospatial_lon_max: -73.79950305175781
    featureType: timeSeries
    dimensions(sizes): obs(24), station(1), station_id_strlen(3)
    variables(dimensions): float64 latitude(station), float64 longitude(station), float64 stationAltitude(station), |S1 station_id(station, station_id_strlen), float64 time(obs), int32 stationIndex(obs), float32 PMSL(obs), float32 ALTI(obs), float32 TMPC(obs), float32 DWPC(obs), float32 SKNT(obs), float32 DRCT(obs), float32 GUST(obs), float32 WNUM(obs), float32 CHC1(obs), float32 CHC2(obs), float32 CHC3(obs)
    groups: 

In [29]:
station_id = data['station_id'][0].tobytes() #get station id
station_id = station_id.decode('ascii')
print(station_id)

ALB


In [30]:
time_var = data.variables['time'] #get the date & time of metar
#print (time_var)
time = num2date(time_var, time_var.units, only_use_cftime_datetimes=False, only_use_python_datetimes=True)
time

masked_array(data=[real_datetime(2024, 2, 22, 18, 40),
                   real_datetime(2024, 2, 22, 20, 0),
                   real_datetime(2024, 2, 22, 21, 0),
                   real_datetime(2024, 2, 22, 22, 0),
                   real_datetime(2024, 2, 22, 23, 0),
                   real_datetime(2024, 2, 23, 1, 0),
                   real_datetime(2024, 2, 23, 2, 0),
                   real_datetime(2024, 2, 23, 3, 0),
                   real_datetime(2024, 2, 23, 4, 0),
                   real_datetime(2024, 2, 23, 5, 0),
                   real_datetime(2024, 2, 23, 6, 0),
                   real_datetime(2024, 2, 23, 7, 20),
                   real_datetime(2024, 2, 23, 8, 20),
                   real_datetime(2024, 2, 23, 9, 20),
                   real_datetime(2024, 2, 23, 9, 40),
                   real_datetime(2024, 2, 23, 10, 0),
                   real_datetime(2024, 2, 23, 11, 20),
                   real_datetime(2024, 2, 23, 12, 20),
                   real_datetim

In [31]:
tmpc = data.variables['TMPC'] #define variables
dwpc = data.variables['DWPC']
#slp = data.variables['PMSL']
wdsp = data.variables['SKNT']
#wdir = data.variables['DRCT']
#gust = data.variables['GUST']
#pres = data.variables['ALTI']

In [32]:
tmpc[20]

masked_array(data=3.3,
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [33]:
length = len(time)
hours = np.arange(0, length, 1)

In [34]:
tmpcs = []
dwpcs = []
wdsps = []
i = 0
for x in hours:
    tmpcs.append(tmpc[i].data) 
    dwpcs.append(dwpc[i].data)
    wdsps.append(wdsp[i].data)
    i = i + 1

In [35]:
tmpCs = tmpcs * units('degC') #attch units where necessary
tmpKs = tmpCs.to('K').magnitude

dwpCs = dwpcs * units('degC')
dwpKs = dwpCs.to('K').magnitude

wdsKt = wdsps * units('kt')
wdmph = wdsKt.to('mph').magnitude

In [36]:
df2 = pd.DataFrame(
    { timeDim : time,
     'Temperature_height_above_ground' : tmpKs,
     'Dewpoint_temperature_height_above_ground' : dwpKs,
     'Wind_speed_height_above_ground' : wdmph}   
)

In [37]:
df2 = df2.set_index(df2.columns[0])

In [38]:
df2

,Temperature_height_above_ground,Dewpoint_temperature_height_above_ground,Wind_speed_height_above_ground
time1,,,
2024-02-22 18:40:00,277.549988,270.350006,14.960134
2024-02-22 20:00:00,278.750000,270.350006,8.055456
2024-02-22 21:00:00,278.750000,270.949982,13.809354
2024-02-22 22:00:00,278.750000,270.949982,11.507795
2024-02-22 23:00:00,279.250000,270.949982,9.206236
2024-02-23 01:00:00,278.750000,270.949982,10.357016
2024-02-23 02:00:00,278.149994,272.549988,12.658574
2024-02-23 03:00:00,277.049988,273.149994,9.206236
2024-02-23 04:00:00,276.449982,273.750000,11.507795


In [39]:
df3 = pd.concat([df2, df])
df3 = df3.rename(columns={timeDim: 'Time',
                          "Temperature_height_above_ground": "T",
                          "Dewpoint_temperature_height_above_ground": "Td",
                          "Wind_speed_height_above_ground": "Wind"})

In [40]:
df3 = df3.reset_index()

In [41]:
df3['Time'] = pd.to_datetime(df3[timeDim])

In [42]:
df3['T'] = (df3['T'] - 273.15) * (9/5) + 32
df3['Td'] = (df3['Td'] - 273.15) * (9/5) + 32

In [43]:
tempsF = df3['T']
windsMph = df3['Wind']

In [44]:
windChills = []
i = 0
for wind in windsMph:
    if tempsF[i] <= 50 and windsMph[i] > 3:
        windChill = 35.74 + (0.6215 * tempsF[i]) - (35.75 * (windsMph[i]**0.16)) + (0.4275 * tempsF[i] * (windsMph[i]**0.16))
        windChills.append(windChill)
    else:
        windChills.append(float('NaN'))
    i = i + 1

In [45]:
df4 = pd.DataFrame(
    {'WindChill' : windChills}   
)

In [46]:
df5 = pd.concat([df3, df4], axis=1)
df5

,time1,T,Td,Wind,Time,WindChill
0,2024-02-22 18:40:00,39.919991,26.960022,14.960134,2024-02-22 18:40:00,31.745513
1,2024-02-22 20:00:00,42.080009,26.960022,8.055456,2024-02-22 20:00:00,37.093573
2,2024-02-22 21:00:00,42.080009,28.039978,13.809354,2024-02-22 21:00:00,34.859980
3,2024-02-22 22:00:00,42.080009,28.039978,11.507795,2024-02-22 22:00:00,35.637173
4,2024-02-22 23:00:00,42.980011,28.039978,9.206236,2024-02-22 23:00:00,37.666212
5,2024-02-23 01:00:00,42.080009,28.039978,10.357016,2024-02-23 01:00:00,36.076071
6,2024-02-23 02:00:00,41.000000,30.919989,12.658574,2024-02-23 02:00:00,33.869472
7,2024-02-23 03:00:00,39.019989,32.000000,9.206236,2024-02-23 03:00:00,32.790227
8,2024-02-23 04:00:00,37.939980,33.080009,11.507795,2024-02-23 04:00:00,30.447780
9,2024-02-23 05:00:00,37.939980,33.080009,6.904677,2024-02-23 05:00:00,32.713682


In [47]:
df5['T'] = round(df5['T'])
df5['Td'] = round(df5['Td'])
df5['Wind'] = round(df5['Wind'])
df5['WindChill'] = round(df5['WindChill'])

In [48]:
df5

,time1,T,Td,Wind,Time,WindChill
0,2024-02-22 18:40:00,40.0,27.0,15.0,2024-02-22 18:40:00,32.0
1,2024-02-22 20:00:00,42.0,27.0,8.0,2024-02-22 20:00:00,37.0
2,2024-02-22 21:00:00,42.0,28.0,14.0,2024-02-22 21:00:00,35.0
3,2024-02-22 22:00:00,42.0,28.0,12.0,2024-02-22 22:00:00,36.0
4,2024-02-22 23:00:00,43.0,28.0,9.0,2024-02-22 23:00:00,38.0
5,2024-02-23 01:00:00,42.0,28.0,10.0,2024-02-23 01:00:00,36.0
6,2024-02-23 02:00:00,41.0,31.0,13.0,2024-02-23 02:00:00,34.0
7,2024-02-23 03:00:00,39.0,32.0,9.0,2024-02-23 03:00:00,33.0
8,2024-02-23 04:00:00,38.0,33.0,12.0,2024-02-23 04:00:00,30.0
9,2024-02-23 05:00:00,38.0,33.0,7.0,2024-02-23 05:00:00,33.0


In [49]:
df5 = df5.drop(columns=timeDim)

In [50]:
df5.to_csv('ALB_obs_fore.csv')